# The Source Coding Theorem

## Information content of independent random variables

If $x$ and $y$ are independent, the following identities hold
$$h\left(x,y\right) = h\left(x\right) + h\left(y\right)$$
$$H\left(X,Y\right) = H\left(X\right) + H\left(Y\right)$$
I.e., entrop and the SHannon information content are additive for independent variables.

## Designing informative experiments

One important property of the entropy is the following: The entropy of an ensemble $X$ is biggest if all the outcomes have equal probabilitiy $p_i=1/|\mathcal{A}_X|$. In other words: The outcome of a random experiment is guaranteed to be most informative if the probability distribution over outcomes is uniform.

### Example: Customer support ticket categories

We want to get a better intuition of what this property means. Let us assume that a support team receives tickets in 4 categories:
* Billing
* Technical issue
* Account access
* Cancellation

If one category happens almost all the time, then each new ticket tells you little (low entropy). If all 4 categories are equally likely, each new ticket is more surprising/informative (high entropy).

Let us start by comparing the entropy of different examples of probability distributions:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from entropy_lab.measures.entropy import compute_entropy

labels = ["Billing", "Tech", "Access", "Cancel"]

distributions = {
    "Very skewed": np.array([0.85, 0.10, 0.03, 0.02]),
    "Moderately skewed": np.array([0.55, 0.20, 0.15, 0.10]),
    "Almost balanced": np.array([0.30, 0.25, 0.25, 0.20]),
    "Uniform (max entropy)": np.array([0.25, 0.25, 0.25, 0.25])
}

for name, p in distributions.items():
    H = compute_entropy(p)
    print(f"{name:22s} -> H = {H:.4f} bits")

As expected, we can observe that:
1. Entropy increases as the distribution becomes more balanced.
2. The uniform case gives the highest entropy.
   
We can visualize this property by generating a plot that moves gradually from a peaked distribution to a uniform one:

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

from entropy_lab.measures.entropy import compute_entropy

p_skewed = np.array([0.85, 0.10, 0.03, 0.02])
p_uniform = np.array([0.25, 0.25, 0.25, 0.25])

alphas = np.linspace(0, 1, 101)
entropies = []

for a in alphas:
    # Interpolate between skewed and uniform
    p = (1 - a) * p_skewed + a * p_uniform
    H = compute_entropy(p, base=2.0)
    entropies.append(H)

plt.figure(figsize=(8, 4.5))
plt.plot(alphas, entropies, linewidth=2)
plt.axhline(np.log2(4), linestyle="--", linewidth=1, label="Max = log2(4) = 2 bits")
plt.xlabel("Balance level (0 = skewed, 1 = uniform)")
plt.ylabel("Entropy (bits)")
plt.title("Entropy is maximal when outcomes are equally likely")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Data compression

We are now convinced that the Shannon information content of an outcome is a natural measure of its information content. In other words, improbable outcomes do convey more information than probable outcomes. Let us now consider how many bits are needed to describe the outcome of an experiment. 

If we can show that we can compress data from a particular source into a file of $L$ bits per source symbol and recover the data reliably, then we will say that the average information content of that source is at most $L$ bits per symbol.

There are several ways of measuring the information content of a random variable. One simple way is to simply count the number of possible outcomes: $|\mathcal{A}_X|$. If we give a binary name to each outcome, in the situation where $|\mathcal{A}_X|$ is a power of 2, the length of each name would be $\log_2|\mathcal{A}_X|$. Therefore:

**The raw bit content** of $X$:
$$H_0\left(X\right) = \log_2|\mathcal{A}_X|$$

$H_0\left(X\right)$ represents the minimum number of bits needed to uniquely label every possible outcome. Let us look at an example:

In [ ]:
import math

# alphabet size of a dice: 6
dice_alphabet_size = 6
raw_bit_content = math.log2(dice_alphabet_size)
raw_bit_content

Using the custom-made function in the ```hartley.py``` file:

In [ ]:
from entropy_lab.measures.hartley import raw_bit_content

raw_bit_content(dice_alphabet_size)

Note that this measure of information content does not include any probabilistic element. What it does is simply to map each outcome to a constant-length binary string. This leads us to the following observation:
1. A *lossy* compressor compresses some files, but maps some files to the *same* encoding. It may be useful in some situations, such as image compression.
2. A *lossless* compressor maps all files to different encodings. 

## Information content defined in terms of lossy compression

We start our exploration by considering lossy compression. Considering the probability of each symbol in an alphabet (for instance, the English ASCII alphabet in an English text), we can observe that some characters occur more frequently than others. To compress such a text, we could imagine to reduce slightly the size of this alphabet by ignoring the less-frequently present characters, such as "!". This means that we would lose the ability to encode some of the more improbable symbols. Therefore, this reduction is associated with a certain *risk*: The larger the risk we are willing to take, the smaller our final alphabet becomes. 

Let us define a new parameter, $\delta$, which represents this risk. Formally, $\delta$ is the probability that there will be no name for an outcome $x$. We can thus define the following:

**The smallest $\delta$-sufficient subset** $S_{\delta}$ is the smallest subset of $\mathcal{A}_X$ satisfying
$$P\left(x\in S_{\delta}\right) \geq 1 - \delta$$

We can construct the subset $S_{\delta}$ by ranking the elements of $\mathcal{A}_X$ in order of decreasing probability and adding successive elements starting from the most probable elements until the total probability is $\geq \left(1-\delta\right)$. An implementation of this algorithm, together with a class to define an alphabet, can be found ```coding/alphabet.py```:

In [ ]:
from collections import Counter 
from entropy_lab.coding.alphabet import AlphabetDistribution

text_example = """
Hello!!! This is an example text... with some rare stuff: @@@ ### $$$
Mostly letters and spaces, but a few symbols appear only once.
"""

counts = Counter(text_example)
counts.pop("\n", None)
counts.pop(" ", None)

dist = AlphabetDistribution.from_counts(dict(counts))

print("Alphabet size:", len(dist.symbols))
print("Top 10 symbols:", dist.ranked().symbols[:10])

delta = 0.05  # allow dropping 5% probability mass
res = dist.delta_subset(delta)

print("\n--- δ-sufficient subset result ---")
print("delta:", delta)
print("threshold (1-delta):", res.threshold)
print("kept alphabet size:", len(res.kept.symbols))
print("kept_probability:", res.kept_probability)
print("dropped_probability:", res.dropped_probability)

print("\nDropped symbols (rare ones):")
print(res.dropped_symbols)

print("\nKept distribution (renormalized) top 10:")
kept_ranked = res.kept.ranked()
for s, p in zip(kept_ranked.symbols[:10], kept_ranked.p[:10]):
    print(repr(s), f"{p:.4f}")